In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/neural-net-nexus-2-0/sample_submission.csv
/kaggle/input/neural-net-nexus-2-0/train.csv
/kaggle/input/neural-net-nexus-2-0/test.csv


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.12/dist-packages/sqlalchemy/orm/query.py:195: SyntaxWarning: "is not" with 'tuple' literal. Did you mean "!="?
  if entities is not ():


In [3]:
train = pd.read_csv('/kaggle/input/neural-net-nexus-2-0/train.csv')
test = pd.read_csv('/kaggle/input/neural-net-nexus-2-0/test.csv')

In [4]:
# Separate features and target
X = train.drop('Revenue', axis=1)
y = train['Revenue']
X = X.drop('ID', axis=1, errors='ignore')


In [5]:
# Advanced feature engineering
def add_advanced_features(df):
    df = df.copy()
    
    # Basic aggregations
    df['Total_Duration'] = df['Administrative_Duration'] + df['Informational_Duration'] + df['ProductRelated_Duration']
    df['Total_Pages'] = df['Administrative'] + df['Informational'] + df['ProductRelated']
    
    # Efficiency metrics
    df['Avg_Duration_Per_Page'] = df['Total_Duration'] / (df['Total_Pages'] + 1)
    df['Product_Ratio'] = df['ProductRelated'] / (df['Total_Pages'] + 1)
    df['Product_Duration_Ratio'] = df['ProductRelated_Duration'] / (df['Total_Duration'] + 1)
    
    # Exit and bounce patterns
    df['Exit_Bounce_Interaction'] = df['ExitRates'] * df['BounceRates']
    df['Exit_Bounce_Diff'] = df['ExitRates'] - df['BounceRates']
    df['Exit_Bounce_Sum'] = df['ExitRates'] + df['BounceRates']
    
    # PageValue interactions (VERY IMPORTANT)
    df['PageValue_Duration'] = df['PageValues'] * df['Total_Duration']
    df['PageValue_Product'] = df['PageValues'] * df['ProductRelated']
    df['PageValue_Exit'] = df['PageValues'] * (1 - df['ExitRates'])
    df['PageValue_Bounce'] = df['PageValues'] * (1 - df['BounceRates'])
    
    # Time-based features
    df['Admin_Duration_Per_Page'] = df['Administrative_Duration'] / (df['Administrative'] + 1)
    df['Info_Duration_Per_Page'] = df['Informational_Duration'] / (df['Informational'] + 1)
    df['Product_Duration_Per_Page'] = df['ProductRelated_Duration'] / (df['ProductRelated'] + 1)
    
    # Engagement scores
    df['Engagement_Score'] = df['PageValues'] * df['ProductRelated'] * (1 - df['BounceRates'])
    df['Quality_Score'] = df['PageValues'] / (df['BounceRates'] + 0.01)
    
    # Interaction depth
    df['Has_Admin'] = (df['Administrative'] > 0).astype(int)
    df['Has_Info'] = (df['Informational'] > 0).astype(int)
    df['Has_Product'] = (df['ProductRelated'] > 0).astype(int)
    df['Page_Type_Diversity'] = df['Has_Admin'] + df['Has_Info'] + df['Has_Product']
    
    # Special day interaction
    df['SpecialDay_PageValue'] = df['SpecialDay'] * df['PageValues']
    df['SpecialDay_Product'] = df['SpecialDay'] * df['ProductRelated']
    
    # Browser/OS/Region interactions with performance
    df['Tech_Exit_Rate'] = df['BounceRates'] * df['OperatingSystems'] / 10
    df['Browser_Engagement'] = df['PageValues'] * df['Browser'] / 10
    
    return df

In [6]:
# Apply feature engineering
X = add_advanced_features(X)
test_X = add_advanced_features(test.drop('ID', axis=1))

In [7]:
# Check imbalance
print("Class distribution:")
print(y.value_counts(normalize=True))
scale_pos_weight = (y == 0).sum() / (y == 1).sum()
print(f"\nScale pos weight: {scale_pos_weight:.2f}")


Class distribution:
Revenue
False    0.844332
True     0.155668
Name: proportion, dtype: float64

Scale pos weight: 5.42


In [8]:
# Define categorical and numerical columns
cat_cols = [
    'Month', 'VisitorType', 'Weekend',
    'OperatingSystems', 'Browser',
    'Region', 'TrafficType'
]
num_cols = [c for c in X.columns if c not in cat_cols]
print(f"\nNumerical features: {len(num_cols)}")
print(f"Categorical features: {len(cat_cols)}")


Numerical features: 35
Categorical features: 7


In [9]:
# Preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', num_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_cols)
    ]
)

In [10]:
# Split for validation
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

# Transform data
X_train_transformed = preprocessor.fit_transform(X_train)
X_val_transformed = preprocessor.transform(X_val)

print("\n" + "="*60)
print("TRAINING MULTIPLE MODELS FOR ENSEMBLE")
print("="*60)



TRAINING MULTIPLE MODELS FOR ENSEMBLE


In [11]:
# Model 1: XGBoost with aggressive regularization
print("\n[1/3] Training XGBoost...")
xgb_model = XGBClassifier(
    n_estimators=1000,
    max_depth=3,              # Even more conservative
    learning_rate=0.02,       # Slower learning
    subsample=0.65,
    colsample_bytree=0.65,
    colsample_bylevel=0.65,
    min_child_weight=7,
    gamma=0.2,
    reg_alpha=0.2,
    reg_lambda=2.0,
    scale_pos_weight=scale_pos_weight,
    eval_metric='auc',
    random_state=42,
    early_stopping_rounds=75,
    n_jobs=-1
)

xgb_model.fit(
    X_train_transformed, y_train,
    eval_set=[(X_val_transformed, y_val)],
    verbose=False
)
xgb_val_probs = xgb_model.predict_proba(X_val_transformed)[:, 1]
xgb_auc = roc_auc_score(y_val, xgb_val_probs)
print(f"XGBoost Validation AUC: {xgb_auc:.4f}")



[1/3] Training XGBoost...
XGBoost Validation AUC: 0.9316


In [12]:
# Model 2: LightGBM
print("\n[2/3] Training LightGBM...")
lgb_model = LGBMClassifier(
    n_estimators=1000,
    max_depth=4,
    learning_rate=0.02,
    subsample=0.7,
    colsample_bytree=0.7,
    min_child_samples=30,
    reg_alpha=0.1,
    reg_lambda=1.5,
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    n_jobs=-1,
    verbose=-1
)

lgb_model.fit(
    X_train_transformed, y_train,
    eval_set=[(X_val_transformed, y_val)],
    callbacks=[],
)
lgb_val_probs = lgb_model.predict_proba(X_val_transformed)[:, 1]
lgb_auc = roc_auc_score(y_val, lgb_val_probs)
print(f"LightGBM Validation AUC: {lgb_auc:.4f}")


[2/3] Training LightGBM...
LightGBM Validation AUC: 0.9252


In [13]:
# Model 3: CatBoost
print("\n[3/3] Training CatBoost...")
cat_model = CatBoostClassifier(
    iterations=1000,
    depth=4,
    learning_rate=0.02,
    l2_leaf_reg=3.0,
    random_seed=42,
    scale_pos_weight=scale_pos_weight,
    verbose=False,
    early_stopping_rounds=75
)

cat_model.fit(
    X_train_transformed, y_train,
    eval_set=(X_val_transformed, y_val),
)
cat_val_probs = cat_model.predict_proba(X_val_transformed)[:, 1]
cat_auc = roc_auc_score(y_val, cat_val_probs)
print(f"CatBoost Validation AUC: {cat_auc:.4f}")


[3/3] Training CatBoost...
CatBoost Validation AUC: 0.9312


In [14]:
# Ensemble predictions with weighted average
print("\n" + "="*60)
print("CREATING WEIGHTED ENSEMBLE")
print("="*60)

# Weight by validation performance
total_auc = xgb_auc + lgb_auc + cat_auc
xgb_weight = xgb_auc / total_auc
lgb_weight = lgb_auc / total_auc
cat_weight = cat_auc / total_auc

print(f"\nModel weights:")
print(f"XGBoost: {xgb_weight:.3f}")
print(f"LightGBM: {lgb_weight:.3f}")
print(f"CatBoost: {cat_weight:.3f}")

ensemble_val_probs = (
    xgb_weight * xgb_val_probs +
    lgb_weight * lgb_val_probs +
    cat_weight * cat_val_probs
)

ensemble_auc = roc_auc_score(y_val, ensemble_val_probs)
print(f"\nEnsemble Validation AUC: {ensemble_auc:.4f}")


CREATING WEIGHTED ENSEMBLE

Model weights:
XGBoost: 0.334
LightGBM: 0.332
CatBoost: 0.334

Ensemble Validation AUC: 0.9311


In [15]:
# Find optimal threshold
print("\n" + "="*60)
print("OPTIMIZING THRESHOLD")
print("="*60)

thresholds = np.arange(0.3, 0.7, 0.02)
best_threshold = 0.5
best_f1 = 0

for thresh in thresholds:
    preds = (ensemble_val_probs >= thresh).astype(int)
    f1 = f1_score(y_val, preds)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = thresh

print(f"Optimal threshold: {best_threshold:.2f}")
print(f"Best F1 Score: {best_f1:.4f}")

val_preds = (ensemble_val_probs >= best_threshold).astype(int)
val_acc = accuracy_score(y_val, val_preds)
print(f"Validation Accuracy: {val_acc:.4f}")


OPTIMIZING THRESHOLD
Optimal threshold: 0.64
Best F1 Score: 0.6851
Validation Accuracy: 0.8963


In [16]:
# Train final models on full data
print("\n" + "="*60)
print("TRAINING FINAL MODELS ON FULL DATA")
print("="*60)

X_transformed = preprocessor.fit_transform(X)
test_X_transformed = preprocessor.transform(test_X)


TRAINING FINAL MODELS ON FULL DATA


In [17]:
# XGBoost final
print("\nTraining final XGBoost...")
xgb_final = XGBClassifier(
    n_estimators=xgb_model.best_iteration,
    max_depth=3,
    learning_rate=0.02,
    subsample=0.65,
    colsample_bytree=0.65,
    colsample_bylevel=0.65,
    min_child_weight=7,
    gamma=0.2,
    reg_alpha=0.2,
    reg_lambda=2.0,
    scale_pos_weight=scale_pos_weight,
    eval_metric='auc',
    random_state=42,
    n_jobs=-1
)
xgb_final.fit(X_transformed, y)


Training final XGBoost...


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=0.65, colsample_bynode=None,
              colsample_bytree=0.65, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='auc', feature_types=None,
              feature_weights=None, gamma=0.2, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.02, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
              max_leaves=None, min_child_weight=7, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=272,
              n_jobs=-1, num_parallel_tree=None, ...)

In [18]:
# LightGBM final
print("Training final LightGBM...")
lgb_final = LGBMClassifier(
    n_estimators=1000,
    max_depth=4,
    learning_rate=0.02,
    subsample=0.7,
    colsample_bytree=0.7,
    min_child_samples=30,
    reg_alpha=0.1,
    reg_lambda=1.5,
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    n_jobs=-1,
    verbose=-1
)
lgb_final.fit(X_transformed, y)

Training final LightGBM...


LGBMClassifier(colsample_bytree=0.7, learning_rate=0.02, max_depth=4,
               min_child_samples=30, n_estimators=1000, n_jobs=-1,
               random_state=42, reg_alpha=0.1, reg_lambda=1.5,
               scale_pos_weight=np.float64(5.423927178153446), subsample=0.7,
               verbose=-1)

In [19]:
# CatBoost final
print("Training final CatBoost...")
cat_final = CatBoostClassifier(
    iterations=cat_model.best_iteration_,
    depth=4,
    learning_rate=0.02,
    l2_leaf_reg=3.0,
    random_seed=42,
    scale_pos_weight=scale_pos_weight,
    verbose=False
)
cat_final.fit(X_transformed, y)

Training final CatBoost...


In [20]:
# Generate ensemble predictions
print("\n" + "="*60)
print("GENERATING FINAL PREDICTIONS")
print("="*60)

xgb_test_probs = xgb_final.predict_proba(test_X_transformed)[:, 1]
lgb_test_probs = lgb_final.predict_proba(test_X_transformed)[:, 1]
cat_test_probs = cat_final.predict_proba(test_X_transformed)[:, 1]

ensemble_test_probs = (
    xgb_weight * xgb_test_probs +
    lgb_weight * lgb_test_probs +
    cat_weight * cat_test_probs
)

# Apply optimal threshold
test_preds = (ensemble_test_probs >= best_threshold).astype(int)


GENERATING FINAL PREDICTIONS


In [21]:
# Create submission
submission = pd.DataFrame({
    'ID': test['ID'],
    'Revenue': test_preds
})

In [22]:
submission.to_csv('submission.csv', index=False)
print("\n✓ Submission file created successfully!")
print("\nPrediction distribution:")
print(submission['Revenue'].value_counts(normalize=True))
print("\nFirst 10 predictions:")
print(submission.head(10))


✓ Submission file created successfully!

Prediction distribution:
Revenue
0    0.808502
1    0.191498
Name: proportion, dtype: float64

First 10 predictions:
   ID  Revenue
0   0        1
1   1        1
2   2        0
3   3        0
4   4        0
5   5        0
6   6        0
7   7        1
8   8        0
9   9        0
